# ML Training - Prédiction de gravité des PASSAGERS

## Contexte du projet

Ce notebook fait partie d'un projet de prédiction de la gravité des accidents routiers en France. L'objectif est de construire un modèle ML capable de prédire si un **usager** (passager, conducteur, piéton) sera **gravement blessé** ou non.

## Objectif spécifique

**Prédire la gravité d'un USAGER** (et non de l'accident global).

La gravité est définie comme :
- `0` = Non grave (indemne ou blessé léger)
- `1` = Grave (hospitalisé ou tué)

## Question clé de ce notebook

**Faut-il entraîner un modèle global ou des modèles spécialisés par catégorie de véhicule ?**

| Approche | Avantages | Inconvénients |
|----------|-----------|---------------|
| **Modèle global** | Plus simple, plus de données | Peut masquer les patterns spécifiques |
| **Modèles par catégorie** | Patterns spécifiques, features adaptées | Moins de données, maintenance multiple |

## Workflow du notebook

```
1. Chargement des données
   ├── Dataset global (594k usagers)
   └── Datasets par catégorie (voiture, 2RM, vélo, piéton, PL)

2. Entraînement XGBoost
   ├── Modèle global
   └── Modèles par catégorie

3. Comparaison des approches
   └── F1, AUC, matrices de confusion

4. Sélection et sauvegarde
   └── Choix de l'approche optimale
```

## Critères de sélection

| Critère | Métrique | Justification |
|---------|----------|---------------|
| **Principal** | F1-Score | Équilibre entre précision et rappel |
| **Secondaire** | AUC | Capacité discriminante |

---

## 1. Configuration et imports

In [1]:
# === IMPORTS ===
import os
import pandas as pd
import numpy as np
import joblib

from ml_config import nb_workers, base_estimators

# Visualisation
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
)

# === CONFIGURATION ===
RANDOM_STATE = 42

print(f"Configuration: {base_estimators} estimators, {nb_workers} workers")
print("Imports OK")

Configuration: 100 estimators, 1 workers
Imports OK


---

## 2. Chargement des données

### Datasets disponibles

| Dataset | Description | Lignes | Taux grave |
|---------|-------------|--------|------------|
| `dataset_passager` | Tous les usagers | ~594k | ~18% |
| `dataset_passager_voiture` | VL, VU, voiturette | ~387k | ~12% |
| `dataset_passager_vehicule_leger` | Motos, scooters, quads | ~101k | ~34% |
| `dataset_passager_velo_edp` | Vélos, trottinettes | ~37k | ~25% |
| `dataset_passager_pieton` | Piétons | ~46k | ~33% |
| `dataset_passager_poids_lourd` | Camions, bus | ~22k | ~6% |

In [2]:
def load_dataset(name, base_path='data/output'):
    """
    Charge un dataset depuis Parquet.
    """
    parquet_path = f'{base_path}/{name}.parquet'
    
    if os.path.exists(parquet_path):
        df = pd.read_parquet(parquet_path)
        print(f'{name}: {df.shape[0]:,} lignes, {df.shape[1]} colonnes')
    else:
        raise FileNotFoundError(f'Dataset {name} non trouvé: {parquet_path}')
    return df

# Chargement du dataset global
print("=== DATASET GLOBAL ===")
df_global = load_dataset('dataset_passager')

# Chargement des datasets par catégorie
print("\n=== DATASETS PAR CATÉGORIE ===")
categories = ['voiture', 'vehicule_leger', 'velo_edp', 'pieton', 'poids_lourd']
datasets = {}

for cat in categories:
    datasets[cat] = load_dataset(f'dataset_passager_{cat}')

print(f"\nTotal par catégorie: {sum(len(df) for df in datasets.values()):,} lignes")

=== DATASET GLOBAL ===
dataset_passager: 594,406 lignes, 18 colonnes

=== DATASETS PAR CATÉGORIE ===
dataset_passager_voiture: 387,387 lignes, 18 colonnes
dataset_passager_vehicule_leger: 101,403 lignes, 18 colonnes
dataset_passager_velo_edp: 37,073 lignes, 16 colonnes
dataset_passager_pieton: 46,080 lignes, 15 colonnes
dataset_passager_poids_lourd: 22,463 lignes, 18 colonnes

Total par catégorie: 594,406 lignes


In [3]:
# Aperçu des colonnes et distributions
print("=== COLONNES DU DATASET GLOBAL ===")
print(df_global.columns.tolist())

print("\n=== DISTRIBUTION DE LA GRAVITÉ ===")
print(f"{'Dataset':<25} {'Lignes':>10} {'Taux grave':>12}")
print("-" * 50)

# Global
taux = df_global['grav_binary'].mean() * 100
print(f"{'Global':<25} {len(df_global):>10,} {taux:>11.1f}%")

# Par catégorie
for cat, df in datasets.items():
    taux = df['grav_binary'].mean() * 100
    print(f"{cat:<25} {len(df):>10,} {taux:>11.1f}%")

=== COLONNES DU DATASET GLOBAL ===
['catu', 'sexe', 'vma', 'dep', 'agg', 'atm', 'catv', 'grav_binary', 'grav_ordered', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'region', 'sexe_conducteur', 'age_conducteur', 'a_equipement_adapte', 'age']

=== DISTRIBUTION DE LA GRAVITÉ ===
Dataset                       Lignes   Taux grave
--------------------------------------------------
Global                       594,406        17.8%
voiture                      387,387        11.8%
vehicule_leger               101,403        33.9%
velo_edp                      37,073        24.6%
pieton                        46,080        32.7%
poids_lourd                   22,463         5.8%


---

## 3. Fonctions utilitaires

### Pipeline de prétraitement

```
Données brutes → Encodage catégorielles → Imputation → Scaling → XGBoost
```

### Gestion du déséquilibre de classes

Les classes sont déséquilibrées (18% grave vs 82% non grave). XGBoost gère cela avec `scale_pos_weight`.

In [4]:
def prepare_data(df, target_col='grav_binary'):
    """
    Prépare les features (X) et la target (y) pour l'entraînement.
    
    - Exclut les colonnes de gravité (évite le data leakage)
    - Encode les variables catégorielles en entiers
    
    Returns: X (DataFrame), y (Series)
    """
    # Colonnes à exclure
    cols_to_drop = ['grav_ordered', 'grav_binary']
    cols_to_drop = [c for c in cols_to_drop if c in df.columns]
    
    X = df.drop(columns=cols_to_drop)
    y = df[target_col]
    
    # Encodage des variables catégorielles
    for col in X.select_dtypes(include=['object', 'string']).columns:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    
    return X, y


def create_xgboost_pipeline(scale_pos_weight=1.0):
    """
    Crée un pipeline sklearn avec XGBoost.
    
    Args:
        scale_pos_weight: ratio négatifs/positifs pour équilibrer les classes
    """
    return Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(
            n_estimators=base_estimators,
            random_state=RANDOM_STATE,
            n_jobs=nb_workers,
            scale_pos_weight=scale_pos_weight,
            eval_metric='logloss'
        ))
    ])


def train_and_evaluate(X_train, X_test, y_train, y_test, model_name):
    """
    Entraîne un modèle XGBoost et calcule les métriques.
    
    Returns:
        - results: dict avec les métriques
        - model: modèle entraîné
        - predictions: dict avec y_pred et y_proba
    """
    # Calcul du scale_pos_weight
    n_neg = (y_train == 0).sum()
    n_pos = (y_train == 1).sum()
    scale_pos_weight = n_neg / n_pos
    
    # Création et entraînement du modèle
    model = create_xgboost_pipeline(scale_pos_weight)
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    # Métriques
    results = {
        'model': model_name,
        'n_train': len(y_train),
        'n_test': len(y_test),
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'auc': roc_auc_score(y_test, y_proba)
    }
    
    predictions = {
        'y_true': y_test,
        'y_pred': y_pred,
        'y_proba': y_proba
    }
    
    return results, model, predictions


def plot_confusion_matrix(y_true, y_pred, model_name):
    """Affiche une matrice de confusion avec Plotly."""
    cm = confusion_matrix(y_true, y_pred)
    labels = ['Non grave (0)', 'Grave (1)']
    
    # Pourcentages
    cm_pct = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    text = [[f"{cm[i][j]}<br>({cm_pct[i][j]:.1f}%)" for j in range(2)] for i in range(2)]
    
    fig = go.Figure(data=go.Heatmap(
        z=cm, x=labels, y=labels,
        text=text, texttemplate="%{text}",
        colorscale='Blues', showscale=True
    ))
    
    fig.update_layout(
        title=f'Matrice de Confusion - {model_name}',
        xaxis_title='Prédiction', yaxis_title='Réalité',
        width=450, height=400
    )
    fig.show()
    return cm


def plot_roc_curve(y_true, y_proba, model_name):
    """Affiche la courbe ROC."""
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines',
                             name=f'ROC (AUC = {roc_auc:.3f})',
                             line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                             name='Random',
                             line=dict(color='gray', dash='dash')))
    
    fig.update_layout(
        title=f'Courbe ROC - {model_name}',
        xaxis_title='Taux Faux Positifs',
        yaxis_title='Taux Vrais Positifs',
        width=500, height=400
    )
    fig.show()
    return roc_auc


print("Fonctions définies")

Fonctions définies


---

## 4. Entraînement sur le dataset GLOBAL

### Approche

Un seul modèle XGBoost entraîné sur **tous les usagers** (~594k lignes).

**Avantages :**
- Maximum de données pour l'apprentissage
- Un seul modèle à maintenir
- Simplicité de déploiement

**Inconvénients :**
- Peut masquer les patterns spécifiques à certaines catégories
- Risque de biais vers les catégories majoritaires (voitures = 65%)

In [5]:
# Préparation des données
X_global, y_global = prepare_data(df_global)

# Split train/test stratifié
X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(
    X_global, y_global,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y_global
)

print(f"Features: {X_global.columns.tolist()}")
print(f"\nTrain: {len(X_train_global):,} | Test: {len(X_test_global):,}")
print(f"Taux grave train: {y_train_global.mean()*100:.1f}%")
print(f"Taux grave test: {y_test_global.mean()*100:.1f}%")

Features: ['catu', 'sexe', 'vma', 'dep', 'agg', 'atm', 'catv', 'est_nuit', 'est_heure_pointe', 'jour_semaine', 'est_weekend', 'region', 'sexe_conducteur', 'age_conducteur', 'a_equipement_adapte', 'age']

Train: 475,524 | Test: 118,882
Taux grave train: 17.8%
Taux grave test: 17.8%


In [6]:
# Entraînement et évaluation
print("Entraînement du modèle GLOBAL...")
results_global, model_global, preds_global = train_and_evaluate(
    X_train_global, X_test_global,
    y_train_global, y_test_global,
    'Global'
)

print(f"\n=== Résultats GLOBAL ===")
print(f"F1-Score : {results_global['f1']:.3f}")
print(f"AUC      : {results_global['auc']:.3f}")
print(f"Precision: {results_global['precision']:.3f}")
print(f"Recall   : {results_global['recall']:.3f}")

Entraînement du modèle GLOBAL...

=== Résultats GLOBAL ===
F1-Score : 0.526
AUC      : 0.842
Precision: 0.395
Recall   : 0.789


In [7]:
# Visualisations
plot_confusion_matrix(preds_global['y_true'], preds_global['y_pred'], 'Global')
plot_roc_curve(preds_global['y_true'], preds_global['y_proba'], 'Global')

0.8422065654150493

---

## 5. Entraînement sur les datasets PAR CATÉGORIE

### Approche

Un modèle XGBoost **spécialisé** pour chaque catégorie de véhicule.

**Catégories :**
| Catégorie | Description | Particularités |
|-----------|-------------|----------------|
| `voiture` | VL, VU | Majorité des données, taux grave bas |
| `vehicule_leger` | 2RM, quads | Usagers vulnérables, taux grave élevé |
| `velo_edp` | Vélos, trottinettes | Pas d'info conducteur (c'est l'usager) |
| `pieton` | Piétons | Pas de véhicule, pas de conducteur |
| `poids_lourd` | Camions, bus | Occupants bien protégés, taux grave bas |

**Avantages :**
- Features adaptées à chaque catégorie
- Patterns spécifiques mieux captés

**Inconvénients :**
- Moins de données par modèle
- 5 modèles à maintenir

In [8]:
# Entraînement sur chaque catégorie
results_categories = []
models_categories = {}
preds_categories = {}

print("=== ENTRAÎNEMENT PAR CATÉGORIE ===")
print()

for cat in categories:
    print(f"--- {cat.upper()} ---")
    
    # Préparation des données
    df_cat = datasets[cat]
    X_cat, y_cat = prepare_data(df_cat)
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_cat, y_cat,
        test_size=0.2,
        random_state=RANDOM_STATE,
        stratify=y_cat
    )
    
    # Entraînement
    results, model, preds = train_and_evaluate(
        X_train, X_test, y_train, y_test, cat
    )
    
    results_categories.append(results)
    models_categories[cat] = model
    preds_categories[cat] = preds
    
    print(f"  Train: {len(X_train):,} | Test: {len(X_test):,}")
    print(f"  F1: {results['f1']:.3f} | AUC: {results['auc']:.3f}")
    print()

=== ENTRAÎNEMENT PAR CATÉGORIE ===

--- VOITURE ---
  Train: 309,909 | Test: 77,478
  F1: 0.423 | AUC: 0.833

--- VEHICULE_LEGER ---
  Train: 81,122 | Test: 20,281
  F1: 0.640 | AUC: 0.795

--- VELO_EDP ---
  Train: 29,658 | Test: 7,415
  F1: 0.557 | AUC: 0.793

--- PIETON ---
  Train: 36,864 | Test: 9,216
  F1: 0.596 | AUC: 0.761

--- POIDS_LOURD ---
  Train: 17,970 | Test: 4,493
  F1: 0.207 | AUC: 0.711



In [9]:
# Tableau récapitulatif
df_results_cat = pd.DataFrame(results_categories)
df_results_cat = df_results_cat[['model', 'n_train', 'n_test', 'accuracy', 'precision', 'recall', 'f1', 'auc']]
print("=== RÉSULTATS PAR CATÉGORIE ===")
print(df_results_cat.round(3).to_string(index=False))

=== RÉSULTATS PAR CATÉGORIE ===
         model  n_train  n_test  accuracy  precision  recall    f1   auc
       voiture   309909   77478     0.749      0.290   0.781 0.423 0.833
vehicule_leger    81122   20281     0.716      0.560   0.746 0.640 0.795
      velo_edp    29658    7415     0.736      0.474   0.676 0.557 0.793
        pieton    36864    9216     0.703      0.537   0.669 0.596 0.761
   poids_lourd    17970    4493     0.838      0.145   0.364 0.207 0.711


In [10]:
# Visualisations pour chaque catégorie
for cat in categories:
    preds = preds_categories[cat]
    print(f"\n{'='*50}")
    print(f"{cat.upper()}")
    print('='*50)
    plot_confusion_matrix(preds['y_true'], preds['y_pred'], cat)


VOITURE



VEHICULE_LEGER



VELO_EDP



PIETON



POIDS_LOURD


---

## 6. Comparaison des approches

### Question clé

Les modèles spécialisés sont-ils significativement meilleurs que le modèle global ?

### Méthode de comparaison

Pour comparer équitablement :
1. **Modèle global** : résultats sur tout le test set
2. **Modèles par catégorie** : résultats agrégés (moyenne pondérée par taille)

### Métriques comparées

- **F1-Score** : critère principal
- **AUC** : départage
- **Recall** : important pour ne pas rater de cas graves

In [11]:
# Calcul des métriques agrégées pour les modèles par catégorie
# Moyenne pondérée par le nombre de samples de test
total_test = sum(r['n_test'] for r in results_categories)

weighted_metrics = {
    'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0, 'auc': 0
}

for r in results_categories:
    weight = r['n_test'] / total_test
    for metric in weighted_metrics:
        weighted_metrics[metric] += r[metric] * weight

# Tableau de comparaison
print("=" * 70)
print("COMPARAISON GLOBAL vs PAR CATÉGORIE")
print("=" * 70)
print()
print(f"{'Approche':<25} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} {'F1':>10} {'AUC':>10}")
print("-" * 75)

# Global
print(f"{'Modèle GLOBAL':<25} {results_global['accuracy']:>10.3f} {results_global['precision']:>10.3f} {results_global['recall']:>10.3f} {results_global['f1']:>10.3f} {results_global['auc']:>10.3f}")

# Par catégorie (agrégé)
print(f"{'Modèles PAR CATÉGORIE':<25} {weighted_metrics['accuracy']:>10.3f} {weighted_metrics['precision']:>10.3f} {weighted_metrics['recall']:>10.3f} {weighted_metrics['f1']:>10.3f} {weighted_metrics['auc']:>10.3f}")

# Différence
print("-" * 75)
diff_f1 = weighted_metrics['f1'] - results_global['f1']
diff_auc = weighted_metrics['auc'] - results_global['auc']
print(f"{'Différence (Cat - Global)':<25} {'-':>10} {'-':>10} {'-':>10} {diff_f1:>+10.3f} {diff_auc:>+10.3f}")

COMPARAISON GLOBAL vs PAR CATÉGORIE

Approche                    Accuracy  Precision     Recall         F1        AUC
---------------------------------------------------------------------------
Modèle GLOBAL                  0.748      0.395      0.789      0.526      0.842
Modèles PAR CATÉGORIE          0.742      0.361      0.744      0.474      0.814
---------------------------------------------------------------------------
Différence (Cat - Global)          -          -          -     -0.053     -0.028


In [12]:
# Graphique de comparaison
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC']
global_values = [results_global['accuracy'], results_global['precision'], 
                 results_global['recall'], results_global['f1'], results_global['auc']]
cat_values = [weighted_metrics['accuracy'], weighted_metrics['precision'],
              weighted_metrics['recall'], weighted_metrics['f1'], weighted_metrics['auc']]

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Modèle GLOBAL',
    x=metrics_names,
    y=global_values,
    marker_color='#3498db',
    text=[f"{v:.3f}" for v in global_values],
    textposition='outside'
))

fig.add_trace(go.Bar(
    name='Modèles PAR CATÉGORIE',
    x=metrics_names,
    y=cat_values,
    marker_color='#e74c3c',
    text=[f"{v:.3f}" for v in cat_values],
    textposition='outside'
))

fig.update_layout(
    title='Comparaison Global vs Par Catégorie (moyenne pondérée)',
    barmode='group',
    yaxis_range=[0, 1.1],
    height=500,
    width=800
)
fig.show()

In [13]:
# Détail par catégorie vs Global
print("=" * 70)
print("DÉTAIL PAR CATÉGORIE")
print("=" * 70)
print()
print(f"{'Catégorie':<20} {'N test':>10} {'F1':>10} {'AUC':>10} {'Taux grave':>12}")
print("-" * 65)

for r in results_categories:
    df_cat = datasets[r['model']]
    taux = df_cat['grav_binary'].mean() * 100
    print(f"{r['model']:<20} {r['n_test']:>10,} {r['f1']:>10.3f} {r['auc']:>10.3f} {taux:>11.1f}%")

print("-" * 65)
print(f"{'GLOBAL':<20} {results_global['n_test']:>10,} {results_global['f1']:>10.3f} {results_global['auc']:>10.3f} {df_global['grav_binary'].mean()*100:>11.1f}%")

DÉTAIL PAR CATÉGORIE

Catégorie                N test         F1        AUC   Taux grave
-----------------------------------------------------------------
voiture                  77,478      0.423      0.833        11.8%
vehicule_leger           20,281      0.640      0.795        33.9%
velo_edp                  7,415      0.557      0.793        24.6%
pieton                    9,216      0.596      0.761        32.7%
poids_lourd               4,493      0.207      0.711         5.8%
-----------------------------------------------------------------
GLOBAL                  118,882      0.526      0.842        17.8%


In [14]:
# Graphique radar par catégorie (F1 et AUC)
fig = make_subplots(rows=1, cols=2, subplot_titles=['F1-Score par catégorie', 'AUC par catégorie'])

cat_names = [r['model'] for r in results_categories]
f1_values = [r['f1'] for r in results_categories]
auc_values = [r['auc'] for r in results_categories]

# F1
fig.add_trace(
    go.Bar(x=cat_names, y=f1_values, marker_color='#2ecc71', name='F1',
           text=[f"{v:.3f}" for v in f1_values], textposition='outside'),
    row=1, col=1
)
fig.add_hline(y=results_global['f1'], line_dash='dash', line_color='red',
              annotation_text=f"Global: {results_global['f1']:.3f}", row=1, col=1)

# AUC
fig.add_trace(
    go.Bar(x=cat_names, y=auc_values, marker_color='#9b59b6', name='AUC',
           text=[f"{v:.3f}" for v in auc_values], textposition='outside'),
    row=1, col=2
)
fig.add_hline(y=results_global['auc'], line_dash='dash', line_color='red',
              annotation_text=f"Global: {results_global['auc']:.3f}", row=1, col=2)

fig.update_layout(height=450, width=1000, showlegend=False)
fig.update_yaxes(range=[0, 1])
fig.show()

---

## 7. Conclusion et recommandation

In [15]:
# Analyse automatique
print("=" * 70)
print("ANALYSE ET RECOMMANDATION")
print("=" * 70)

# Comparaison
if diff_f1 > 0.01:  # Seuil de 1% d'amélioration
    recommendation = "PAR CATÉGORIE"
    reason = f"Les modèles par catégorie offrent un gain de F1 de {diff_f1*100:.1f}%"
elif diff_f1 < -0.01:
    recommendation = "GLOBAL"
    reason = f"Le modèle global est meilleur de {-diff_f1*100:.1f}% en F1"
else:
    recommendation = "GLOBAL"
    reason = "Les performances sont similaires, le modèle global est plus simple"

print(f"\nRecommandation : Approche {recommendation}")
print(f"Justification  : {reason}")

print("\n--- Résumé des performances ---")
print(f"Modèle GLOBAL        : F1 = {results_global['f1']:.3f} | AUC = {results_global['auc']:.3f}")
print(f"Modèles PAR CATÉGORIE: F1 = {weighted_metrics['f1']:.3f} | AUC = {weighted_metrics['auc']:.3f}")

# Analyse par catégorie
print("\n--- Analyse par catégorie ---")
for r in results_categories:
    if r['f1'] > results_global['f1'] + 0.02:
        print(f"{r['model']}: Modèle spécialisé NETTEMENT meilleur (+{(r['f1']-results_global['f1'])*100:.1f}% F1)")
    elif r['f1'] < results_global['f1'] - 0.02:
        print(f"{r['model']}: Modèle spécialisé MOINS bon ({(r['f1']-results_global['f1'])*100:.1f}% F1) - manque de données ?")
    else:
        print(f"{r['model']}: Performances similaires au global")

ANALYSE ET RECOMMANDATION

Recommandation : Approche GLOBAL
Justification  : Le modèle global est meilleur de 5.3% en F1

--- Résumé des performances ---
Modèle GLOBAL        : F1 = 0.526 | AUC = 0.842
Modèles PAR CATÉGORIE: F1 = 0.474 | AUC = 0.814

--- Analyse par catégorie ---
voiture: Modèle spécialisé MOINS bon (-10.3% F1) - manque de données ?
vehicule_leger: Modèle spécialisé NETTEMENT meilleur (+11.4% F1)
velo_edp: Modèle spécialisé NETTEMENT meilleur (+3.1% F1)
pieton: Modèle spécialisé NETTEMENT meilleur (+6.9% F1)
poids_lourd: Modèle spécialisé MOINS bon (-31.9% F1) - manque de données ?


---

## 8. Sauvegarde des modèles

In [16]:
# Création du dossier models
os.makedirs('models', exist_ok=True)

# Sauvegarde du modèle global
joblib.dump(model_global, 'models/model_passager_global.joblib')
joblib.dump(X_global.columns.tolist(), 'models/features_passager_global.joblib')
print("Sauvegardé: model_passager_global.joblib")

# Sauvegarde des modèles par catégorie
for cat, model in models_categories.items():
    joblib.dump(model, f'models/model_passager_{cat}.joblib')
    # Sauvegarder aussi les features utilisées
    X_cat, _ = prepare_data(datasets[cat])
    joblib.dump(X_cat.columns.tolist(), f'models/features_passager_{cat}.joblib')
    print(f"Sauvegardé: model_passager_{cat}.joblib")

# Sauvegarde des résultats pour comparaison ultérieure
all_results = {
    'global': results_global,
    'categories': results_categories,
    'weighted_metrics': weighted_metrics,
    'recommendation': recommendation
}
joblib.dump(all_results, 'models/results_passager_comparison.joblib')
print("\nSauvegardé: results_passager_comparison.joblib")

Sauvegardé: model_passager_global.joblib
Sauvegardé: model_passager_voiture.joblib
Sauvegardé: model_passager_vehicule_leger.joblib
Sauvegardé: model_passager_velo_edp.joblib
Sauvegardé: model_passager_pieton.joblib
Sauvegardé: model_passager_poids_lourd.joblib

Sauvegardé: results_passager_comparison.joblib


---

## Résumé

### Modèles sauvegardés

| Fichier | Description |
|---------|-------------|
| `model_passager_global.joblib` | XGBoost sur tous les usagers |
| `model_passager_voiture.joblib` | XGBoost spécialisé voitures |
| `model_passager_vehicule_leger.joblib` | XGBoost spécialisé 2RM |
| `model_passager_velo_edp.joblib` | XGBoost spécialisé vélos/EDP |
| `model_passager_pieton.joblib` | XGBoost spécialisé piétons |
| `model_passager_poids_lourd.joblib` | XGBoost spécialisé PL |

### Prochaines étapes

1. **Fine-tuning** (notebook 4-b) : Optimiser les hyperparamètres avec Hyperopt
2. **Déploiement API** : Choisir l'approche recommandée pour l'endpoint
3. **Monitoring** : Suivre les performances en production par catégorie